### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Polycystic_Ovary_Syndrome/GSE151158'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = True
trait_row = 11
age_row = 1
gender_row = 2

def convert_trait(value):
    if value is None:
        return None
    val = value.split(':')[-1].strip()
    return 1 if val.upper() == 'Y' else 0

def convert_age(value):
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return value  # Directly return if already numeric
    val = value.split(':')[-1].strip()
    try:
        return float(val)
    except ValueError:
        return None

def convert_gender(value):
    if value is None:
        return None
    val = value.split(':')[-1].strip().upper()
    if val == 'F':
        return 0
    elif val == 'M':
        return 1
    else:
        return None

save_cohort_info('GSE151158', './preprocessed/Polycystic_Ovary_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

selected_clinical_data = geo_select_clinical_features(clinical_data, 'Polycystic_Ovary_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Polycystic_Ovary_Syndrome/trait_data/GSE151158.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = False


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

# Since requires_gene_mapping is False, we skip STEP6 and proceed to the next step.


import os

# Import necessary variables and tools probably defined/obtained in previous steps
csv_path = './preprocessed/Polycystic_Ovary_Syndrome/trait_data/GSE151158.csv'
if 'selected_clinical_data' not in locals():
    if os.path.exists(csv_path):
        selected_clinical_data = pd.read_csv(csv_path, index_col=0)
    else:
        raise FileNotFoundError(f"Required file not found: {csv_path}")

# Ensure directories exist before attempting to save files
os.makedirs(os.path.dirname('./preprocessed/Polycystic_Ovary_Syndrome/gene_data/GSE151158.csv'), exist_ok=True)
os.makedirs(os.path.dirname('./preprocessed/Polycystic_Ovary_Syndrome/trait_data/GSE151158.csv'), exist_ok=True)

# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Polycystic_Ovary_Syndrome/gene_data/GSE151158.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data are severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Polycystic_Ovary_Syndrome')

# 4. Save the cohort information.
json_path = './preprocessed/Polycystic_Ovary_Syndrome/cohort_info.json'
save_cohort_info('GSE151158', json_path, True, True, trait_biased, unbiased_merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Polycystic_Ovary_Syndrome/GSE151158.csv'
    unbiased_merged_data.to_csv(csv_path)
